# Create db from scraped data

## Imports

In [1]:
import glob
import json
import re

import networkx as nx
from itertools import combinations

## Read raw API call files

Reads all the stored API call files and compiles them into a more useful DataFrame.

In [18]:
user_map = {}
transaction_map = {}

for f_name in tqdm(glob.glob('data/raw/*.json')):
    with open(f_name, 'r') as f:
        data = json.load(f)
        for entry in data['data']:
            actor = entry['actor']
            
            assert(len(entry['transactions']) == 1)
            transaction = entry['transactions'][0]
            target = transaction['target']
            if (target in ['a phone number', 'an email', None] or
                target.get('name') == 'a user on iMessage'):
                continue
            
            # custom entries
            entry['from'] = int(actor['id'])
            entry['to'] = int(target['id'])

            user_map[int(actor['id'])] = actor
            user_map[int(target['id'])] = target
            transaction_map[entry['story_id']] = entry

HBox(children=(IntProgress(value=0, max=30360), HTML(value='')))

In [19]:
crawled_files = glob.glob('data/raw/*.json')
crawled_users = set([int(re.match('data/raw/([0-9]*)-[0-9]{3}[.]json', fname).groups()[0]) 
                     for fname in crawled_files])
del crawled_files

In [87]:
transactions = pd.DataFrame().from_dict(transaction_map, orient='index')[[
     'created_time',
     'from',
     'to',
     'message',
     'type',
     'payment_id',
     'permalink',
     'via',
     'story_id',
     'updated_time',
     'audience',
]]
transactions['created_time'] = pd.to_datetime(transactions.created_time)
transactions['updated_time'] = pd.to_datetime(transactions.updated_time)

users = pd.DataFrame().from_dict(user_map, orient='index')[[
    'username',
     'name',
     'firstname',
     'lastname',
     'date_created',
     'cancelled',
     'external_id',
     'is_business',
     'picture',
]]
users['date_created'] = pd.to_datetime(users.date_created)

users.loc[crawled_users,'is_crawled'] = True
users.is_crawled.fillna(False, inplace=True)

transactions.to_csv('data/processed/transactions.csv', index_label='Id')
users.to_csv('data/processed/users.csv', index_label='Id')

In [86]:
# save full dataset
transactions.to_csv('site/data/transactions.csv', index_label='Id')
users.to_csv('site/data/users.csv', index_label='Id')

In [85]:
# save filtered dataset
users.query('is_crawled').to_csv('site/data/users.csv', index_label='Id')
crawled_ids = set(users.query('is_crawled').index)

transactions[transactions['from'].isin(crawled_ids) & 
             transactions['to'].isin(crawled_ids)].to_csv('site/data/transactions.csv', index_label='Id')

## Figure out new IDs to crawl

In [21]:
harvard_names = pd.concat([pd.read_csv(f) for f in glob.glob('data/harvard/*.csv')],
                          sort=False) \
    .reset_index(drop=True)

In [22]:
harvard_names['name'] = harvard_names['First Name'] + ' ' + harvard_names['Last Name']

In [23]:
len(harvard_names)

4681

In [24]:
len(harvard_names.name.unique())

4659

In [25]:
len(users.query('is_crawled').name.unique())

3571

In [26]:
len(crawled_users)

3926

In [37]:
len(transactions)

381613

In [27]:
prominence = transactions.groupby('from').size().add(transactions.groupby('to').size(), fill_value=0)

In [28]:
to_crawl = users.loc[list(prominence[users.name.isin(harvard_names.name) & 
                       (~prominence.index.isin(crawled_users))]
    .sort_values(ascending=False).index),:]

In [29]:
to_crawl_ids = list(to_crawl.index)

Store this list of values to crawl to hand off to the crawling notebook.

In [30]:
%store to_crawl_ids

Stored 'to_crawl_ids' (list)


In [31]:
len(to_crawl)

212

In [33]:
len(to_crawl.name.unique())

133

In [428]:
len(harvard_names)

3379

## Inspect and play around with the data

In [77]:
name = 'Rachel Share-Sapolsky'
u_id = users[users.name == name].index[0]
user_transacts = transactions[(transactions['from'] == u_id) | 
                              (transactions.to == u_id)]

counts = (user_transacts['from']
    .where(user_transacts['from'] != u_id, user_transacts.to)
    .to_frame('other')
    .groupby('other')
    .size())

(pd.concat([counts.to_frame('count'), users.name], axis=1)
    .sort_values('count', ascending=True)
    .dropna()
    .set_index('name')
    .iloc[-10:]
    .plot
    .barh())

plt.grid()
plt.legend()
ft_ax(title="%s's Venmo transactions" % name, subtitle='As of 2018-08-04', y=1.1, left_axis=True)

[]

In [76]:
user_transacts\
.merge(users.loc[:,['name']], left_on='from', right_index=True) \
.rename(columns={'name': 'from_name'}) \
.merge(users.loc[:,['name']], left_on='to', right_index=True) \
.rename(columns={'name': 'to_name'}) \
.loc[:,['created_time', 'message', 'from_name', 'to_name']]

created_time  \
56d8b7dbcd03c9af22178511 2016-03-03 22:16:59   
56da498fcd03c9af2234ba8b 2016-03-05 02:50:55   
570d775bcd03c9af228f366e 2016-04-12 22:31:55   
572d54d6cd03c9af22d62462 2016-05-07 02:37:10   
573325c6cd03c9af22382524 2016-05-11 12:29:58   
589ff48723e064eac05e01e1 2017-02-12 05:37:11   
59121002a385c7e9b5f12288 2017-05-09 18:52:50   
56f9913dcd03c9af22274735 2016-03-28 20:17:01   
572815a9cd03c9af227385a0 2016-05-03 03:06:17   
576610d8cd03c9af22adb8a3 2016-06-19 03:26:16   
598547fd5a877f6e3891700c 2017-08-05 04:22:21   
59c7c72e5a877f6e381cd470 2017-09-24 14:54:38   
59c7c7305a877f6e381d0e9c 2017-09-24 14:54:40   
577095c023e064eac05a71ac 2016-06-27 02:56:00   
57bdf80623e064eac02388cf 2016-08-24 19:39:50   
582fdbaa23e064eac02dc879 2016-11-19 04:57:14   
58770a5623e064eac080be92 2017-01-12 04:47:18   
5877d48f23e064eac08df397 2017-01-12 19:10:07   
5a49542b5a877f6e382d43dc 2017-12-31 21:18:35   
5b11db7a5a877f6e383e9373 2018-06-01 23:49:14   
5774604223e064eac0994d66 2016-06-29 23:56:49   
5775d1a323e064eac0b72eae 2016-07-01 02:12:51   
578c2f3323e064eac06a7dbe 2016-07-18 01:21:55   
596a515f5a877f6e381a26c1 2017-07-15 17:31:11   
5983efa05a877f6e38bb32b3 2017-08-04 03:53:04   
5a49541d5a877f6e382ba4ee 2017-12-31 21:18:21   
5790573323e064eac0b10dff 2016-07-21 05:01:39   
57905c4823e064eac0b147d3 2016-07-21 05:23:20   
598548d45a877f6e38a526cf 2017-08-05 04:25:56   
57b6767923e064eac08a5910 2016-08-19 03:01:13   
...                                      ...   
597521c55a877f6e387bf347 2017-07-23 22:23:01   
598ff5c95a877f6e3802cfec 2017-08-13 06:46:33   
5961c1e90f7b403c8b5ddb0c 2017-07-09 05:40:57   
599113965a877f6e38cffc26 2017-08-14 03:05:58   
59af45125a877f6e389dbc43 2017-09-06 00:45:06   
59b82afd5a877f6e38c54246 2017-09-12 18:44:13   
59d12de25a877f6e3886a770 2017-10-01 18:03:13   
5a9e5aa75a877f6e38d57915 2018-03-06 09:08:55   
5abbecc35a877f6e38325cd4 2018-03-28 19:28:03   
5b1c84fd5a877f6e381fa9f0 2018-06-10 01:55:09   
5b2d896127064c70e4abc3bd 2018-06-22 23:42:24   
5b52799ce1f606dc2ff47cc7 2018-07-21 00:09:00   
5a57f62f5a877f6e38eadf99 2018-01-11 23:41:34   
5a9312735a877f6e38607a50 2018-02-25 19:45:55   
5a96cde45a877f6e388d75d9 2018-02-28 15:42:28   
5b0e393c5a877f6e381c367a 2018-05-30 05:40:12   
5a80bbff5a877f6e38a9f683 2018-02-11 21:56:15   
5a80bcae5a877f6e38c405dd 2018-02-11 21:59:10   
5ab05fcc5a877f6e3807979e 2018-03-20 01:11:40   
5b425875e1f606dc2f161248 2018-07-08 18:31:17   
5b454f6ce1f606dc2fa6c90e 2018-07-11 00:29:32   
5b527991e1f606dc2ff2b6e9 2018-07-21 00:08:49   
5b56834de1f606dc2f185b70 2018-07-24 01:39:25   
5b941486e1f606dc2f0f8611 2018-09-08 18:27:18   
5b957fd8e1f606dc2f9acd1b 2018-09-09 20:17:28   
5bac041ee1f606dc2f7e9284 2018-09-26 22:11:42   
5be5e88de1f606dc2f62e9b1 2018-11-09 20:05:33   
5ba7ef63e1f606dc2f27225d 2018-09-23 19:54:11   
5bafff96e1f606dc2f560edb 2018-09-29 22:41:26   
5be4db98e1f606dc2f5848bf 2018-11-09 00:57:58   

                                                                    message  \
56d8b7dbcd03c9af22178511                                         keychange🔑   
56da498fcd03c9af2234ba8b                                      pizza pizza 🍕   
570d775bcd03c9af228f366e                                          ♻️🚯☃🔮🕳↪️🔀   
572d54d6cd03c9af22d62462                                love and friendship   
573325c6cd03c9af22382524                                            tissues   
589ff48723e064eac05e01e1                                       cookie cream   
59121002a385c7e9b5f12288                                            stunted   
56f9913dcd03c9af22274735                                                  🎫   
572815a9cd03c9af227385a0                                             formal   
576610d8cd03c9af22adb8a3                        hanging out without @phong1   
598547fd5a877f6e3891700c                                               uber   
59c7c72e5a877f6e381cd470                                                fud   
59c7c7305a877f6e381d0e9c    